# Import Model

In [1]:
from cameo import models
bigg_list = models.index_models_bigg()

/home/matinnu/anaconda3/envs/lab_rotation_3/lib/python3.7/site-packages/sympy/__init__.py:676: SymPyDeprecationWarning: 

importing sympy.core.singleton with 'from sympy import *' has been
deprecated since SymPy 1.6. Use import sympy.core.singleton instead.
See https://github.com/sympy/sympy/issues/18245 for more info.

  deprecated_since_version="1.6").warn()


In [2]:
# Find GEMS that starts with "Eschericia coli" and ends with "MG1655"
e_coli_mg1655 = bigg_list[bigg_list.organism.str.contains("^Escherichia coli.*MG1655$")]
e_coli_mg1655

,bigg_id,gene_count,reaction_count,organism,metabolite_count
0,e_coli_core,137,95,Escherichia coli str. K-12 substr. MG1655,72
2,iAF1260,1261,2382,Escherichia coli str. K-12 substr. MG1655,1668
3,iAF1260b,1261,2388,Escherichia coli str. K-12 substr. MG1655,1668
71,iJO1366,1367,2583,Escherichia coli str. K-12 substr. MG1655,1805
72,iJR904,904,1075,Escherichia coli str. K-12 substr. MG1655,761
76,iML1515,1516,2712,Escherichia coli str. K-12 substr. MG1655,1877


In [3]:
model_id = e_coli_mg1655.iloc[e_coli_mg1655.reaction_count.argmax()].bigg_id
model_id

'iML1515'

# Simulate Model

In [4]:
from cameo import load_model
model = load_model(model_id)
model

Using license file /home/matinnu/gurobi.lic
Academic license - for non-commercial use only


Name,iML1515
Memory address,0x07f1198cb0950
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [5]:
model.solver = 'gurobi'

In [6]:
from cameo import fba
%time fba_result = fba(model)

CPU times: user 422 ms, sys: 46.9 ms, total: 469 ms
Wall time: 459 ms


In [7]:
fba_result.data_frame

,flux
CYTDK2,0.000000
XPPT,0.000000
HXPRT,0.000000
NDPK5,-0.022948
SHK3Dr,0.334240
...,...
MPTS,0.000000
MOCOS,0.000000
BMOGDS2,0.000000
FESD2s,0.000000


In [8]:
#fba_result.display_on_map("iJO1366.Central metabolism")

In [9]:
fba_result.fluxes

CYTDK2     0.000000
XPPT       0.000000
HXPRT      0.000000
NDPK5     -0.022948
SHK3Dr     0.334240
             ...   
MPTS       0.000000
MOCOS      0.000000
BMOGDS2    0.000000
FESD2s     0.000000
OCTNLL     0.000000
Name: fluxes, Length: 2712, dtype: float64

In [10]:
import escher
from escher import Builder

In [11]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    reaction_data=fba_result.fluxes,
)
builder

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': -0.022947509112696166, 'SHK3Dr': 0.3…

In [12]:
# Open List of Knocknout Genes


In [13]:
# Create Knockout - Acetic acid producer
# Create Knockout - Acetic acid auxotroph
# Test the Knockout - producer
# Test the Knockout - auxotroph
# Test dFBA
# Create coculture model